In [43]:
# !pip install scikit-image
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# from sklearn.model_selection import train_test_split
from PIL import Image, ImageCms
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [44]:
# photos_path = './faces/Train/'
# test_path = './faces/Test/'
photos_path = './imagenet/pomegranate/train'
test_path = './imagenet/pomegranate/test'
# photos_path = './unsplash_photos/train'
# test_path = './unsplash_photos/test'

RGB = 'RGB'
LAB = 'LAB'
train_split = 0.8
random_seed = 42
# For faces and unsplash
# resolution = (256,256)
# For imagenet
resolution = (64, 64)


# Converter for Lab colourspace
srgb_p = ImageCms.createProfile("sRGB")
lab_p  = ImageCms.createProfile("LAB")

rgb2lab_PIL = ImageCms.buildTransformFromOpenProfiles(srgb_p, lab_p, "RGB", "LAB")
lab2rgb_PIL = ImageCms.buildTransformFromOpenProfiles(lab_p, srgb_p, "LAB", "RGB")

In [45]:
photos = []

# using PIL
# for photo in os.listdir(photos_path):
#     rgb_photo = Image.open(os.path.join(photos_path, photo)).resize(resolution).convert(RGB)
#     lab_photo =  ImageCms.applyTransform(rgb_photo, rgb2lab)
#     lab_array = np.array(lab_photo)
#     photos.append(lab_array)

# Using Scikit Image
for photo_name in os.listdir(photos_path):
    photo = load_img(os.path.join(photos_path, photo_name))
    photo = photo.resize(resolution)
    photo_array = img_to_array(photo)
    photos.append(photo_array)

In [46]:
photos_np = np.array(photos, dtype= float)
# photos_np /= 255.0

split_len = int(len(photos_np) * train_split)

photos_train = photos_np[:split_len]
photos_train /= 255.0

photos_val = photos_np[split_len:]

In [47]:
photos_np.shape

(481, 64, 64, 3)

In [54]:
neurons_set_1 = [64, 128, 256, 512]
neurons_set_2 = [256, 128]
# neurons_set_2 = [256, 128, 64]
upsampling_neurons = [128, 64, 32, 16]
filter_size = (3,3)
upsampling_filter_size = (2,2)
activation_hidden = 'relu'
activation_output = 'tanh'
strides = 2

epochs = 15
steps_per_epoch = 1000

# create model
model = Sequential()

# Input layer
model.add(InputLayer(input_shape= resolution + (1,)))

# Hidden layer set 1
for neuron in neurons_set_1:
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same'))
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same', strides=strides))
    model.add(BatchNormalization())
    
# Hidden layer set 2
for neuron in neurons_set_2:
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same'))
    model.add(BatchNormalization())

# Upsampling Hidden layer
for neuron in upsampling_neurons:
    model.add(UpSampling2D(upsampling_filter_size))
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same'))
    model.add(BatchNormalization())

# prepare output layer
model.add(Conv2D(2, (3,3), activation=activation_output, padding='same'))
# model.add(UpSampling2D(upsampling_filter_size))

model.compile(optimizer = 'rmsprop', loss='mse')


In [55]:
lab_val = rgb2lab(photos_val/255.0)
X_val = lab_val[:,:,:,0]
X_val = X_val.reshape(X_val.shape + (1,))

y_val = lab_val[:,:,:,1:]
y_val /= 128

In [56]:
x = rgb2lab(photos_train[0])
np.min(x[:,:,0]), np.max(x[:,:,0]), np.min(x[:,:,1:]), np.max(x[:,:,1:])

(1.4470766912513042, 98.35009388875834, -9.190135242181551, 57.63323037027413)

In [57]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 64, 64, 64)        640       
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_36 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
batch_normalization_37 (Batc (None, 16, 16, 128)       512       
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 16, 16, 256)      

In [58]:
# Image transformer
datagen = ImageDataGenerator(
    shear_range=0.2, zoom_range=0.2, rotation_range=20, horizontal_flip=True)

# Generate training data
batch_size = 64
def batch_generator(batch_size):
    for photos_batch in datagen.flow(photos_train, batch_size=batch_size):
        # scikit-image
        LAB_images = rgb2lab(photos_batch)
        X_batch = LAB_images[:, :, :, 0]
        Y_batch = LAB_images[:, :, :, 1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)
        # PIL
        # X_batch = photos_batch[:,:,:,0]
        # y_batch = photos_batch[:,:,:,1:]
        # y_batch -= 128
        # y_batch /= 128
        # yield (X_batch.reshape(X_batch.shape + (1,)), y_batch)


# Train model
model_name = "imagenet_pomeg_1000"
tensorboard = TensorBoard(log_dir="output/"+model_name)
checkpoint_filepath = './checkpoint/'+model_name
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)
history = model.fit(batch_generator(batch_size), callbacks=[tensorboard, model_checkpoint_callback], epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data= (X_val,y_val))


Epoch 1/15
1000/1000 [==============================] - 218s 211ms/step - loss: 0.0153 - val_loss: 0.0239
Epoch 2/15
1000/1000 [==============================] - 218s 218ms/step - loss: 0.0060 - val_loss: 0.0238
Epoch 3/15
1000/1000 [==============================] - 220s 220ms/step - loss: 0.0050 - val_loss: 0.0230
Epoch 4/15
1000/1000 [==============================] - 246s 246ms/step - loss: 0.0044 - val_loss: 0.0215
Epoch 5/15
1000/1000 [==============================] - 254s 254ms/step - loss: 0.0041 - val_loss: 0.0327
Epoch 6/15
1000/1000 [==============================] - 190s 190ms/step - loss: 0.0038 - val_loss: 0.0241
Epoch 7/15
1000/1000 [==============================] - 189s 189ms/step - loss: 0.0037 - val_loss: 0.0233
Epoch 8/15
1000/1000 [==============================] - 185s 185ms/step - loss: 0.0035 - val_loss: 0.0232
Epoch 9/15
1000/1000 [==============================] - 184s 184ms/step - loss: 0.0033 - val_loss: 0.0218
Epoch 10/15
1000/1000 [=======================

In [59]:
print(model.evaluate(X_val, y_val, batch_size=batch_size))

2/2 [==============================] - 1s 1s/step - loss: 0.0219
0.021895894780755043


In [60]:
testing_set = []

# For PIL
# for photo_name in os.listdir(test_path):
#     rgb_photo = Image.open(os.path.join(test_path, photo_name)).resize(resolution).convert(RGB)
#     lab_photo =  ImageCms.applyTransform(rgb_photo, rgb2lab)
#     lab_array = np.array(lab_photo)
#     testing_set.append(lab_array)

# for Scikit image
for photo_name in os.listdir(test_path):
    photo = load_img(os.path.join(test_path, photo_name))
    photo = photo.resize(resolution)
    photo_array = img_to_array(photo)
    testing_set.append(photo_array)

testing_set = np.array(testing_set, dtype = float)
lab_test = rgb2lab(testing_set/255.0)

test_photos = lab_test[:,:,:,0]
test_photos = test_photos.reshape(test_photos.shape + (1,))

output = model.predict(test_photos)
output *= 128
output = output.astype(int)

output_path = f"Result/imagenet_pomeg_{epochs}_{steps_per_epoch}/"

# Check whether the specified path exists or not
exists = os.path.exists(output_path)
if not exists:
   # Create a new directory because it does not exist
   os.makedirs(output_path)
   print("The new directory is created!")
   
testing_set = testing_set.astype(int)
for i in range(len(output)):
   canvas = np.zeros(resolution + (3,))
   bw_part = testing_set[i][:,:,0]
   canvas[:,:,0] = bw_part
   canvas[:,:,1:] = output[i]
   print(i, np.max(output[i]),np.min(output[i]), np.min(testing_set[i]), np.max(testing_set[i]))
   rgb_canvas = lab2rgb(canvas)
     
   # using PIL
   #  lab_image = Image.fromarray(canvas, mode="LAB")
   #  rgb_image = ImageCms.applyTransform(lab_image, lab2rgb)
   #  rgb_image.save(output_path+f"output_{i}.jpeg")

   # using scikit image
   imsave(output_path+f"output_{i}.jpeg", lab2rgb(canvas))


d:\ProgramData\Anaconda3\envs\kewlar\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return func(*args, **kwargs)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
d:\ProgramData\Anaconda3\envs\kewlar\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 144 pixels
  return func(*args, **kwargs)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Conve

The new directory is created!
0 52 -5 0 255
1 59 -36 0 255
2 58 -23 0 255
3 52 -8 0 255
4 57 -12 0 255
5 61 -12 0 255
6 47 -23 0 255
7 50 -11 0 255
8 62 -24 0 237
9 39 -28 0 255
10 48 -7 0 255
11 53 -26 0 255
12 50 -7 0 255
13 38 -16 0 255
14 49 -11 0 255


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


15 41 -9 0 255
16 44 -18 0 255
17 50 -7 0 255
18 46 -7 0 255


In [61]:
# Save model
model_json = model.to_json()
with open(f"{model_name}_{epochs}.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(f"{model_name}_{epochs}.h5")